In [1]:
from __future__ import division, print_function, absolute_import

seed_num = 72

import numpy as np
np.random.seed(seed_num)
import random
random.seed(seed_num)
import tensorflow as tf
tf.set_random_seed(seed_num)

import os
import sys
import glob
import math
from functools import reduce
import operator as op
from sklearn.utils import class_weight
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

dataset_folder = os.path.abspath("./individual_npzs/{0}/*.npz")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

np.set_printoptions(suppress=True)

## Parameters

In [2]:
learning_rate = 0.0001
batch_size = 64
dropout = 0.75
max_pool = 2
strides = 1
input_size = 60000
output_size = 4
epochs = 1000
timesteps = 38

hidden_layer = 128
l2_beta = 0.0001
tolerance = 50

validation_session = 1
test_session = 2

label_dictionary = {'ang': 0, 'hap': 1, 'neu': 2, 'sad': 3}
onehot_dictionary = {0: 'ang', 1: 'hap', 2:'neu', 3:'sad'}

## Helper methods

In [3]:
def batch_generator(data, labels):
    steps = math.ceil(data.shape[0] / batch_size)
    for batch_step in range(0, steps):
        start = batch_size * batch_step
        end = batch_size * (batch_step + 1)
        yield data[start:end], labels[start:end], batch_step
        
def build_encoded_array(emotion_label):
    initialized_array = [0. for key in label_dictionary]
    initialized_array[label_dictionary[emotion_label]] = 1.
    return initialized_array
        
def onehot_encode(label_minibatch):
    return [build_encoded_array(emotion_label) for emotion_label in label_minibatch]

def compute_class_weights(labels):
    return np.ndarray.tolist(class_weight.compute_class_weight('balanced', np.unique(labels), labels))

def sparse_encode(label_minibatch):
    return [label_dictionary[emotion_label] for emotion_label in label_minibatch]

def onehot_decode(onehot_labels):
    return [onehot_dictionary[label.index(1.0)] for label in onehot_labels]

def weighted_accuracy(y_pred, y_true):
    #Suma vsetkych spravne urcenych viet predelena vsetkymi vetami

    correct = 0
    all = 0
    for key in y_true:
        correct += y_pred[key]
        all += y_true[key]

    return correct/all


def unweighted_accuracy(y_pred, y_true):

    #Suma poctu spravnych viet lomeno vsetky vety pre kazdu emociu (triedu) predelena poctom tried

    correct = 0
    emotions = len(y_true)
    for key in y_true:
        correct += y_pred[key]/y_true[key]

    return correct/emotions

def validation_results(current_sess, data, labels):
    predictions = current_sess.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled_validation = list(zip(predictions, onehot_decode(labels)))
    final_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}
    overall_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}

    for couple in coupled_validation:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    unweighted_acc = unweighted_accuracy(final_dict, overall_dict)
    return unweighted_acc

## Load dataset: 3 sessions for training, 1 for validation, 1 for test

In [4]:
train_dataset = []
validation_dataset = []
test_dataset = []

train_labels = []
validation_labels = []
test_labels = []

all_sessions = {1: [], 2:[], 3:[], 4:[], 5:[]}

session_string = 'session{0}'

for i in range(1, 6):
    formatted = session_string.format(i)
    for spectrogram in glob.glob(dataset_folder.format(formatted)):
        loaded_spec = np.load(spectrogram)
        for x in loaded_spec['spectrograms']:
            if i != validation_session and i != test_session:
                train_dataset.append(x) 
            elif i == validation_session:
                validation_dataset.append(x)
            elif i == test_session:
                test_dataset.append(x)
        for x in loaded_spec['labels']:
            all_sessions[i].append(x)
            if i != validation_session and i != test_session:
                train_labels.append(x) 
            elif i == validation_session:
                validation_labels.append(x)
            elif i == test_session:
                test_labels.append(x)
        
train_dataset = np.asarray(train_dataset)
train_labels = np.asarray(train_labels)

validation_dataset = np.asarray(validation_dataset)
validation_labels = np.asarray(validation_labels)

test_dataset = np.asarray(test_dataset)
test_labels = np.asarray(test_labels)

In [5]:
train_data = np.zeros([len(train_dataset), train_dataset[0].shape[0], train_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(train_dataset)):
    train_data[data,:,:] = train_dataset[data]
    
validation_data = np.zeros([len(validation_dataset), validation_dataset[0].shape[0], validation_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(validation_dataset)):
    validation_data[data,:,:] = validation_dataset[data]
    
test_data = np.zeros([len(test_dataset), test_dataset[0].shape[0], test_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(test_dataset)):
    test_data[data,:,:] = test_dataset[data]

In [6]:
concatted = np.concatenate((train_labels, test_labels, validation_labels))
counts = np.unique(train_labels, return_counts=True)[1]
sum_counts = sum(counts)
class_weights = [(0.25/(count/sum_counts)) for count in counts]

In [7]:
class_weights

[1.74609375, 1.8687290969899666, 0.5808212058212058, 0.8543577981651376]

In [8]:
np.unique(validation_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([ 84,  46, 275, 173], dtype=int64))

In [9]:
np.unique(test_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([ 48,  81, 422, 193], dtype=int64))

In [10]:
validation_data = validation_data.reshape((validation_data.shape[0], input_size))
validation_labels = onehot_encode(validation_labels)

test_data = test_data.reshape((test_data.shape[0], input_size))
test_labels = onehot_encode(test_labels)

train_data = train_data.reshape((train_data.shape[0], input_size))
train_labels = onehot_encode(train_labels)

In [11]:
train_data, train_labels = shuffle(train_data, train_labels, random_state=seed_num)

## Convolutional

In [12]:
def conv2d(to_process, weights, biases, strides=1):
    conv_out = tf.nn.conv2d(to_process, weights, strides=[1, strides, strides, 1], padding='SAME')
    bias_out = tf.nn.bias_add(conv_out, biases)
    relu_out = tf.nn.relu(bias_out)
    return relu_out

def maxpool2d(to_pool, pool_size=2):
    maxpool_out = tf.nn.max_pool(to_pool, ksize=[1, pool_size, pool_size, 1], strides=[1, pool_size, pool_size, 1], padding='SAME')
    return maxpool_out

def nn_pipeline(spectrogram, weights, biases):
    
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 300, 1])
    
    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, pool_size=2)
    
    second_layer_out = conv2d(first_maxpool_out, weights['second_layer_weights'], biases['second_layer_biases'])
    second_maxpool_out = maxpool2d(second_layer_out, pool_size=2)
    
    third_layer_out = conv2d(second_maxpool_out, weights['third_layer_weights'], biases['third_layer_biases'])
    third_maxpool_out = maxpool2d(third_layer_out, pool_size=2)
    
    reshape_for_fc = tf.reshape(third_maxpool_out, [-1, weights['fully_connected_weights'].get_shape().as_list()[0]])
    fully_connected_out = tf.add(tf.matmul(reshape_for_fc, weights['fully_connected_weights']), biases['fully_connected_biases'])
    fully_connected_activation = tf.nn.relu(fully_connected_out)
    fully_connected_dropout = tf.nn.dropout(fully_connected_activation, dropout)
    
    fully_connected_out_2 = tf.add(tf.matmul(fully_connected_dropout, weights['fully_connected_weights_2']), biases['fully_connected_biases_2'])
    fully_connected_activation_2 = tf.nn.relu(fully_connected_out_2)
    fully_connected_dropout_2 = tf.nn.dropout(fully_connected_activation_2, dropout)
    
    prediction = tf.add(tf.matmul(fully_connected_dropout_2, weights['output']), biases['output'])
    
    return prediction

## Testing

In [13]:
def nn_pipeline_rnn(spectrogram, weights, biases, dropout_num, normalization_switch):
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 300, 1])

    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, pool_size=2)
    first_batch_norm = tf.layers.batch_normalization(first_maxpool_out, training=normalization_switch)

    second_layer_out = conv2d(first_batch_norm, weights['second_layer_weights'], biases['second_layer_biases'])
    second_maxpool_out = maxpool2d(second_layer_out, pool_size=2)
    second_batch_norm = tf.layers.batch_normalization(second_maxpool_out, training=normalization_switch)

    third_layer_out = conv2d(second_batch_norm, weights['third_layer_weights'], biases['third_layer_biases'])
    third_maxpool_out = maxpool2d(third_layer_out, pool_size=2)
    third_batch_norm = tf.layers.batch_normalization(third_maxpool_out, training=normalization_switch)

    interim_shape = third_batch_norm.get_shape().as_list()
    transposed = tf.transpose(third_batch_norm, perm=[0, 2, 1, 3])
    reshape_for_rnn = tf.reshape(transposed, [-1, interim_shape[2], interim_shape[1]*interim_shape[3]])
    reshape_for_rnn.set_shape([None, interim_shape[2], interim_shape[1]*interim_shape[3]])

    #hidden_list = [hidden_layer, hidden_layer]

    #gru_fw_cell = [tf.contrib.rnn.GRUCell(hidden) for hidden in hidden_list]
    #gru_bw_cell = [tf.contrib.rnn.GRUCell(hidden) for hidden in hidden_list]
    
    fw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    bw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    gru_fw_cell = tf.contrib.rnn.DropoutWrapper(cell=fw_cell, output_keep_prob=dropout_num, seed=seed_num)
    gru_bw_cell = tf.contrib.rnn.DropoutWrapper(cell=bw_cell, output_keep_prob=dropout_num, seed=seed_num)
    
    #gru_output, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(gru_fw_cell, gru_bw_cell, reshape_for_rnn, dtype=tf.float32)
    gru_output, _ = tf.nn.bidirectional_dynamic_rnn(gru_fw_cell, gru_bw_cell, reshape_for_rnn, dtype=tf.float32)
    gru_output_concatted = tf.concat(gru_output, axis=2)
    interim_shape_gru = tf.shape(gru_output_concatted)
    gru_flatten = tf.reshape(gru_output_concatted, [-1, interim_shape_gru[1]*interim_shape_gru[2]])
    
    fully_connected_out = tf.add(tf.matmul(gru_flatten, weights['gru_weights']), biases['gru_biases'])
    fully_connected_activation = tf.nn.relu(fully_connected_out)
    fully_connected_dropout = tf.nn.dropout(fully_connected_activation, dropout_num, seed=seed_num)
    
    prediction = tf.add(tf.matmul(fully_connected_dropout, weights['output']), biases['output'])
        
    return prediction

In [14]:
weights = {
    'first_layer_weights': tf.Variable(tf.random_normal([10, 15, 1, 16])),
    'second_layer_weights': tf.Variable(tf.random_normal([8, 10, 16, 24])),
    'third_layer_weights': tf.Variable(tf.random_normal([5, 8, 24, 32])),
    'fully_connected_weights': tf.Variable(tf.random_normal([25*38*32, 2048])),
    'fully_connected_weights_2': tf.Variable(tf.random_normal([2048, 2048])),
    'output': tf.Variable(tf.random_normal([2048, output_size]))
}

biases = {
    'first_layer_biases': tf.Variable(tf.random_normal([16])),
    'second_layer_biases': tf.Variable(tf.random_normal([24])),
    'third_layer_biases': tf.Variable(tf.random_normal([32])),
    'fully_connected_biases': tf.Variable(tf.random_normal([2048])),
    'fully_connected_biases_2': tf.Variable(tf.random_normal([2048])),
    'output': tf.Variable(tf.random_normal([output_size]))
}

In [15]:
weights_rnn = {
    'first_layer_weights': tf.Variable(tf.truncated_normal([10, 15, 1, 16], seed=seed_num)),
    'second_layer_weights': tf.Variable(tf.truncated_normal([8, 10, 16, 24], seed=seed_num)),
    'third_layer_weights': tf.Variable(tf.truncated_normal([5, 8, 24, 32], seed=seed_num)),
    'gru_weights': tf.Variable(tf.truncated_normal([2*hidden_layer*timesteps, hidden_layer], seed=seed_num)),
    'output': tf.Variable(tf.truncated_normal([hidden_layer, output_size], seed=seed_num))
}

biases_rnn = {
    'first_layer_biases': tf.Variable(tf.truncated_normal([16], seed=seed_num)),
    'second_layer_biases': tf.Variable(tf.truncated_normal([24], seed=seed_num)),
    'third_layer_biases': tf.Variable(tf.truncated_normal([32], seed=seed_num)),
    'gru_biases': tf.Variable(tf.truncated_normal([hidden_layer], seed=seed_num)),
    'output': tf.Variable(tf.truncated_normal([output_size], seed=seed_num))
}

In [16]:
def loss_function(logits, labels):
    flat_logits = tf.reshape(logits, [-1, output_size])
    flat_labels = tf.reshape(labels, [-1, output_size])
    
    eps = tf.constant(value=1e-10)
    flat_logits = flat_logits + eps
    
    softmax = tf.nn.softmax(flat_logits)
    
    coeffs = tf.constant(class_weights)
    
    cross_entropy = -tf.reduce_sum(tf.multiply(flat_labels * tf.log(softmax + eps), coeffs), reduction_indices=[1])
    
    l2_loss = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * l2_beta
    
    return tf.reduce_mean(tf.add(cross_entropy, l2_loss))

In [17]:
X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, output_size])

keep_prob = tf.placeholder(tf.float32)
normalization_switch = tf.placeholder(tf.bool)

logits = nn_pipeline_rnn(X, weights_rnn, biases_rnn, keep_prob, normalization_switch)
prediction = tf.nn.softmax(logits)

#weighted_logits = tf.multiply(class_weights, logits)
#weight_regularizer = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * 0.01
#loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

loss_function = loss_function(logits, Y)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_trigger = optimizer.minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [18]:
with tf.device('/gpu:0'):
    sess = tf.Session()
    sess.run(init)
    
    best_acc = 0
    best_loss = sys.maxsize
    best_train_acc = 0
    current_tolerance = 0
    
    print('Session initialized.')

    for epoch_step in range(1, epochs+1):
        batch_gen = batch_generator(train_data, train_labels)
        train_acc_all = []
        train_loss_all = []
        for data_minibatch, label_minibatch, current_index in batch_gen:
            sess.run(train_trigger, feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: dropout, normalization_switch: True})
            train_loss, train_acc = sess.run([loss_function, accuracy], feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: 1.0, normalization_switch: False})
            train_acc_all.append(train_acc)
            train_loss_all.append(train_loss)
        train_acc_whole = sum(train_acc_all)/len(train_acc_all)
        train_loss_whole = sum(train_loss_all)/len(train_loss_all)
        print("Training accuracy and loss of epoch #" + str(epoch_step) + ": {:.4f}".format(train_acc_whole) + ", {:.4f}".format(train_loss_whole))
        loss, acc = sess.run([loss_function, accuracy], feed_dict={X: validation_data, Y: validation_labels, keep_prob: 1.0, normalization_switch: False})
        unweighted_acc = validation_results(sess, validation_data, validation_labels)
        print("Validation after epoch #" + str(epoch_step) + ", Validation Loss= "+ "{:.4f}".format(loss) + ", Validation Accuracy= " + "{:.3f}".format(acc) + ", Unweighted Accuracy= " + "{:.3f}".format(unweighted_acc))
        saved_by = []
        if acc > best_acc:
            best_acc = acc
            saver.save(sess, "./saved_models/best_model_bidirectional_128_l2_accuracy_1000eps.ckpt")
            saved_by.append(" accuracy ")
        if loss < best_loss:
            best_loss = loss
            saver.save(sess, "./saved_models/best_model_bidirectional_128_l2_loss_1000eps.ckpt")
            saved_by.append(" loss")
        if len(saved_by) > 0:
            current_tolerance = 0
            print("Saved model by {0} at: {1} epoch with {2}, {3}, {4}".format(saved_by, epoch_step, best_loss, acc, unweighted_acc))
        else:
            current_tolerance += 1
        if current_tolerance == tolerance:
            print("Tolerance reached, stopping early. Not really")
        if train_acc_whole > best_train_acc:
            best_train_acc = train_acc_whole
            saver.save(sess, "./saved_models/best_model_bidirectional_128_l2_accuracy_1000eps_training.ckpt")
            print("Saved model by train acc {0}".format(train_acc_whole))
            
      
    
    print("Optimization Finished!")

    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={X: test_data,
    #                                  Y: test_labels,
    #                                  keep_prob: 1.0, normalization_switch: False}))

Session initialized.
Training accuracy and loss of epoch #1: 0.4155, 67.6588
Validation after epoch #1, Validation Loss= 65.6719, Validation Accuracy= 0.448, Unweighted Accuracy= 0.257
Saved model by [' accuracy ', ' loss'] at: 1 epoch with 65.67192077636719, 0.4480968713760376, 0.25690852625635235
Saved model by train acc 0.4154509689126696
Training accuracy and loss of epoch #2: 0.4014, 67.4362
Validation after epoch #2, Validation Loss= 65.5794, Validation Accuracy= 0.420, Unweighted Accuracy= 0.276
Saved model by [' loss'] at: 2 epoch with 65.57935333251953, 0.4204152226448059, 0.275795404227171
Training accuracy and loss of epoch #3: 0.3868, 66.9887
Validation after epoch #3, Validation Loss= 65.0171, Validation Accuracy= 0.394, Unweighted Accuracy= 0.296
Saved model by [' loss'] at: 3 epoch with 65.01713562011719, 0.3944636583328247, 0.2963815088739693
Training accuracy and loss of epoch #4: 0.3926, 66.5246
Validation after epoch #4, Validation Loss= 64.8986, Validation Accuracy=

Training accuracy and loss of epoch #41: 0.4374, 62.3433
Validation after epoch #41, Validation Loss= 62.7806, Validation Accuracy= 0.394, Unweighted Accuracy= 0.396
Training accuracy and loss of epoch #42: 0.4284, 62.4225
Validation after epoch #42, Validation Loss= 63.1875, Validation Accuracy= 0.355, Unweighted Accuracy= 0.389
Training accuracy and loss of epoch #43: 0.4273, 62.5473
Validation after epoch #43, Validation Loss= 62.8441, Validation Accuracy= 0.394, Unweighted Accuracy= 0.391
Training accuracy and loss of epoch #44: 0.4560, 61.9914
Validation after epoch #44, Validation Loss= 62.4890, Validation Accuracy= 0.407, Unweighted Accuracy= 0.419
Saved model by [' loss'] at: 44 epoch with 62.48897171020508, 0.40657439827919006, 0.4191221689845716
Saved model by train acc 0.45597003613199505
Training accuracy and loss of epoch #45: 0.4604, 61.8261
Validation after epoch #45, Validation Loss= 62.5993, Validation Accuracy= 0.396, Unweighted Accuracy= 0.403
Saved model by train ac

Validation after epoch #79, Validation Loss= 61.8744, Validation Accuracy= 0.464, Unweighted Accuracy= 0.438
Training accuracy and loss of epoch #80: 0.5695, 59.7773
Validation after epoch #80, Validation Loss= 62.2653, Validation Accuracy= 0.448, Unweighted Accuracy= 0.425
Training accuracy and loss of epoch #81: 0.5668, 59.8249
Validation after epoch #81, Validation Loss= 62.1558, Validation Accuracy= 0.465, Unweighted Accuracy= 0.431
Training accuracy and loss of epoch #82: 0.5748, 59.7722
Validation after epoch #82, Validation Loss= 62.5198, Validation Accuracy= 0.441, Unweighted Accuracy= 0.405
Saved model by train acc 0.5748335344450815
Training accuracy and loss of epoch #83: 0.5609, 59.9038
Validation after epoch #83, Validation Loss= 62.6014, Validation Accuracy= 0.415, Unweighted Accuracy= 0.423
Training accuracy and loss of epoch #84: 0.5385, 59.9890
Validation after epoch #84, Validation Loss= 62.1536, Validation Accuracy= 0.455, Unweighted Accuracy= 0.437
Training accuracy

Validation after epoch #122, Validation Loss= 61.0687, Validation Accuracy= 0.488, Unweighted Accuracy= 0.473
Training accuracy and loss of epoch #123: 0.6026, 58.7389
Validation after epoch #123, Validation Loss= 61.0098, Validation Accuracy= 0.493, Unweighted Accuracy= 0.471
Training accuracy and loss of epoch #124: 0.6093, 58.6215
Validation after epoch #124, Validation Loss= 61.2118, Validation Accuracy= 0.483, Unweighted Accuracy= 0.469
Training accuracy and loss of epoch #125: 0.6021, 58.7297
Validation after epoch #125, Validation Loss= 61.1228, Validation Accuracy= 0.490, Unweighted Accuracy= 0.476
Training accuracy and loss of epoch #126: 0.5986, 58.8380
Validation after epoch #126, Validation Loss= 61.1751, Validation Accuracy= 0.465, Unweighted Accuracy= 0.471
Training accuracy and loss of epoch #127: 0.5682, 59.1791
Validation after epoch #127, Validation Loss= 61.2654, Validation Accuracy= 0.472, Unweighted Accuracy= 0.469
Training accuracy and loss of epoch #128: 0.5937, 

Training accuracy and loss of epoch #168: 0.6720, 57.7117
Validation after epoch #168, Validation Loss= 61.1352, Validation Accuracy= 0.500, Unweighted Accuracy= 0.466
Training accuracy and loss of epoch #169: 0.6455, 57.6946
Validation after epoch #169, Validation Loss= 61.1474, Validation Accuracy= 0.491, Unweighted Accuracy= 0.466
Training accuracy and loss of epoch #170: 0.6612, 57.6117
Validation after epoch #170, Validation Loss= 61.1363, Validation Accuracy= 0.507, Unweighted Accuracy= 0.470
Training accuracy and loss of epoch #171: 0.6773, 57.6815
Validation after epoch #171, Validation Loss= 61.1930, Validation Accuracy= 0.503, Unweighted Accuracy= 0.464
Training accuracy and loss of epoch #172: 0.6697, 57.6672
Validation after epoch #172, Validation Loss= 61.1780, Validation Accuracy= 0.498, Unweighted Accuracy= 0.466
Training accuracy and loss of epoch #173: 0.6720, 57.6427
Validation after epoch #173, Validation Loss= 61.2732, Validation Accuracy= 0.493, Unweighted Accuracy

...

Training accuracy and loss of epoch #783: 0.9338, 51.9862
Validation after epoch #783, Validation Loss= 61.0671, Validation Accuracy= 0.522, Unweighted Accuracy= 0.471
Training accuracy and loss of epoch #784: 0.9329, 52.0093
Validation after epoch #784, Validation Loss= 60.8879, Validation Accuracy= 0.522, Unweighted Accuracy= 0.474
Training accuracy and loss of epoch #785: 0.9347, 51.9868
Validation after epoch #785, Validation Loss= 61.0022, Validation Accuracy= 0.517, Unweighted Accuracy= 0.470
Training accuracy and loss of epoch #786: 0.9334, 52.0081
Validation after epoch #786, Validation Loss= 61.0287, Validation Accuracy= 0.517, Unweighted Accuracy= 0.469
Training accuracy and loss of epoch #787: 0.9307, 52.0507
Validation after epoch #787, Validation Loss= 60.8947, Validation Accuracy= 0.517, Unweighted Accuracy= 0.478
Training accuracy and loss of epoch #788: 0.9267, 52.0995
Validation after epoch #788, Validation Loss= 60.9458, Validation Accuracy= 0.509, Unweighted Accuracy

Training accuracy and loss of epoch #831: 0.9441, 51.8972
Validation after epoch #831, Validation Loss= 61.0629, Validation Accuracy= 0.510, Unweighted Accuracy= 0.457
Training accuracy and loss of epoch #832: 0.9459, 51.8636
Validation after epoch #832, Validation Loss= 61.1470, Validation Accuracy= 0.516, Unweighted Accuracy= 0.458
Saved model by train acc 0.9458686436925615
Training accuracy and loss of epoch #833: 0.9432, 51.8926
Validation after epoch #833, Validation Loss= 61.0481, Validation Accuracy= 0.521, Unweighted Accuracy= 0.459
Training accuracy and loss of epoch #834: 0.9468, 51.8632
Validation after epoch #834, Validation Loss= 61.1179, Validation Accuracy= 0.522, Unweighted Accuracy= 0.454
Saved model by train acc 0.9467615008354187
Training accuracy and loss of epoch #835: 0.9454, 51.8844
Validation after epoch #835, Validation Loss= 60.8469, Validation Accuracy= 0.533, Unweighted Accuracy= 0.468
Training accuracy and loss of epoch #836: 0.9427, 51.8966
Validation aft

Training accuracy and loss of epoch #879: 0.9463, 51.8408
Validation after epoch #879, Validation Loss= 61.2478, Validation Accuracy= 0.514, Unweighted Accuracy= 0.463
Training accuracy and loss of epoch #880: 0.9450, 51.8349
Validation after epoch #880, Validation Loss= 61.2631, Validation Accuracy= 0.503, Unweighted Accuracy= 0.458
Training accuracy and loss of epoch #881: 0.9468, 51.8338
Validation after epoch #881, Validation Loss= 61.2614, Validation Accuracy= 0.512, Unweighted Accuracy= 0.460
Training accuracy and loss of epoch #882: 0.9468, 51.8274
Validation after epoch #882, Validation Loss= 61.5271, Validation Accuracy= 0.512, Unweighted Accuracy= 0.455
Training accuracy and loss of epoch #883: 0.9490, 51.7959
Validation after epoch #883, Validation Loss= 60.9399, Validation Accuracy= 0.531, Unweighted Accuracy= 0.467
Training accuracy and loss of epoch #884: 0.9463, 51.8079
Validation after epoch #884, Validation Loss= 61.0240, Validation Accuracy= 0.526, Unweighted Accuracy

Training accuracy and loss of epoch #927: 0.9517, 51.7505
Validation after epoch #927, Validation Loss= 61.2170, Validation Accuracy= 0.516, Unweighted Accuracy= 0.467
Training accuracy and loss of epoch #928: 0.9530, 51.7211
Validation after epoch #928, Validation Loss= 61.0149, Validation Accuracy= 0.533, Unweighted Accuracy= 0.468
Training accuracy and loss of epoch #929: 0.9539, 51.7373
Validation after epoch #929, Validation Loss= 61.2315, Validation Accuracy= 0.514, Unweighted Accuracy= 0.455
Saved model by train acc 0.9539043579782759
Training accuracy and loss of epoch #930: 0.9544, 51.7164
Validation after epoch #930, Validation Loss= 61.2307, Validation Accuracy= 0.517, Unweighted Accuracy= 0.458
Saved model by train acc 0.9543507865497044
Training accuracy and loss of epoch #931: 0.9530, 51.7342
Validation after epoch #931, Validation Loss= 60.8116, Validation Accuracy= 0.529, Unweighted Accuracy= 0.476
Training accuracy and loss of epoch #932: 0.9517, 51.7403
Validation aft

Training accuracy and loss of epoch #975: 0.9535, 51.6908
Validation after epoch #975, Validation Loss= 60.5650, Validation Accuracy= 0.535, Unweighted Accuracy= 0.487
Training accuracy and loss of epoch #976: 0.9548, 51.6762
Validation after epoch #976, Validation Loss= 60.4843, Validation Accuracy= 0.542, Unweighted Accuracy= 0.487
Training accuracy and loss of epoch #977: 0.9561, 51.6603
Validation after epoch #977, Validation Loss= 60.3931, Validation Accuracy= 0.547, Unweighted Accuracy= 0.501
Saved model by train acc 0.9561365008354187
Training accuracy and loss of epoch #978: 0.9561, 51.6804
Validation after epoch #978, Validation Loss= 60.4871, Validation Accuracy= 0.547, Unweighted Accuracy= 0.501
Training accuracy and loss of epoch #979: 0.9575, 51.6523
Validation after epoch #979, Validation Loss= 60.6440, Validation Accuracy= 0.536, Unweighted Accuracy= 0.484
Saved model by train acc 0.9574757865497044
Training accuracy and loss of epoch #980: 0.9570, 51.6621
Validation aft

In [19]:
sess.close()

In [44]:
sess = tf.Session()

saver.restore(sess, "./saved_models/best_model_bidirectional_128_l2_accuracy_1000eps.ckpt")

INFO:tensorflow:Restoring parameters from ./saved_models/best_model_bidirectional_128_l2_accuracy_1000eps_training.ckpt


In [45]:
def get_confusion_matrix(session, data, labels):
    predictions = sess.run(tf.argmax(prediction, 1), feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    true_labels = [onehot.index(1.0) for onehot in labels]
    return confusion_matrix(predictions, true_labels)

def get_final_results(session, data, labels):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    predictions = session.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled = list(zip(predictions, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

def get_final_results_glued(session, labels, correct):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    coupled = list(zip(correct, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

In [46]:
get_confusion_matrix(sess, validation_data, validation_labels)

array([[ 45,   5,   9,   2],
       [  3,   7,   7,   3],
       [ 28,  20, 113,  23],
       [  8,  14, 146, 145]], dtype=int64)

In [47]:
final_dict_validation, overall_dict_validation = get_final_results(sess, validation_data, validation_labels)

In [48]:
print(unweighted_accuracy(final_dict_validation, overall_dict_validation))
print(weighted_accuracy(final_dict_validation, overall_dict_validation))

0.4842368946710489
0.5363321799307958


In [49]:
final_preds = np.asarray([])
final_correct_preds = np.asarray([])
final_probs = np.empty((0, 4))


midpoint = math.ceil(len(test_labels)/2)
for i in range(0, 2):
    predictions_test = sess.run(correct_prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    probabilities_test = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    predictions_test_confusion = sess.run(tf.argmax(prediction, 1), feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    final_correct_preds = np.concatenate((final_correct_preds, predictions_test))
    final_probs = np.concatenate((final_probs, probabilities_test))
    final_preds = np.concatenate((final_preds, predictions_test_confusion))

In [50]:
confusion_matrix(final_preds, [onehot.index(1.0) for onehot in test_labels])

array([[ 14,  14,  20,   3],
       [  7,  16,  50,   3],
       [ 20,  40, 244,  35],
       [  7,  11, 108, 152]], dtype=int64)

In [51]:
final_dict_test, overall_dict_test = get_final_results_glued(sess, test_labels, final_correct_preds)

In [52]:
print(final_dict_test)
print(overall_dict_test)

{'ang': 14, 'hap': 16, 'neu': 244, 'sad': 152}
{'ang': 48, 'hap': 81, 'neu': 422, 'sad': 193}


In [53]:
print(unweighted_accuracy(final_dict_test, overall_dict_test))
print(weighted_accuracy(final_dict_test, overall_dict_test))

0.4637403374590693
0.5725806451612904


In [54]:
print(precision_recall_fscore_support(final_preds, [onehot.index(1.0) for onehot in test_labels], average='macro')[0:2])

(0.4637403374590693, 0.4378906803596275)


In [59]:
np.set_printoptions(threshold=np.nan)

print(final_correct_preds)

[0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.